## Lab 1: シンプルなカスタマーサポートエージェントプロトタイプの作成

### 概要

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/) は、AI エージェントを安全かつスケーラブルにデプロイ・運用するためのサービスです。任意のフレームワークやモデルを使用でき、プロトタイプから本番環境への移行を迅速に行うことができます。

この5つのLabで構成されるチュートリアルでは、**カスタマーサポートエージェント**を使用して、プロトタイプから本番環境までのエンドツーエンドの開発プロセスを実演します。この例では、[Strands Agents](https://strandsagents.com/latest/)（シンプルで使いやすいコードファーストのエージェント構築フレームワーク）と Amazon Bedrock の Anthropic Claude Haiku 4.5 モデルを使用します。お客様のアプリケーションでは、任意のフレームワークやモデルを選択できます。ここで説明する概念は、他のフレームワークやモデルでも同様に適用できることに注意してください。

**ワークショップの流れ:**
- **Lab 1（現在）**: エージェントプロトタイプの作成 - 機能するカスタマーサポートエージェントを構築
- **Lab 2**: Memory による強化 - 会話コンテキストとパーソナライゼーションを追加
- **Lab 3**: Gateway と Identity によるスケーリング - エージェント間でツールを安全に共有
- **Lab 4**: 本番環境へのデプロイ - AgentCore Runtime を使用してオブザーバビリティを実現
- **Lab 5**: エージェントパフォーマンスの評価 - オンライン評価で品質を監視
- **Lab 6**: ユーザーインターフェースの構築 - 顧客向けアプリケーションを作成

この最初のLabでは、ワークショップ全体を通じて進化し、永続的なMemory、共有ツール、完全なオブザーバビリティを備えた本番環境対応システムになるカスタマーサポートエージェントのプロトタイプを構築します。エージェントには以下のローカルツールが利用可能です：
- **get_return_policy()** - 特定の製品の返品ポリシーを取得
- **get_product_info()** - 製品情報を取得
- **web_search()** - トラブルシューティングのヘルプをウェブで検索
- **get_technical_support()** - Bedrock Knowledge Base を検索


### Lab 1 のアーキテクチャ
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*ローカルで実行するシンプルなプロトタイプ。以降のLabでは、これを共有ツール、永続的なMemory、本番環境グレードのオブザーバビリティを備えた AgentCore サービスに移行します。*

### 前提条件

* 適切な権限を持つ **AWS アカウント**
* ローカルにインストールされた **Python 3.10+**
* 認証情報が設定された **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) で有効化された **Anthropic Claude 3.7**
* 次のセルでインストールする **Strands Agents** およびその他のライブラリ

#### AWS ワークショップアカウントを使用していない場合

セルフペースラボとして実行する場合は、CloudFormation スタックを作成してデプロイするために追加の2つのステップを実行する必要があります：

**ステップ 0.1: [セルフペースラボのみ]** 

以下のコマンド `!aws sts get-caller-identity` を含むセルのコメントを解除して、SageMaker ロールを確認してください。AWS コンソールで IAM に移動し、SageMaker ロールを検索します。次に、[ワークショップのセルフペース前提条件](https://catalog.us-east-1.prod.workshops.aws/workshops/850fcd5c-fd1f-48d7-932c-ad9babede979/en-US/00-prerequisites/02-self-paced) Labで説明されているように、[IAM ポリシー、AWS マネージドポリシー、信頼関係](https://catalog.us-east-1.prod.workshops.aws/workshops/850fcd5c-fd1f-48d7-932c-ad9babede979/en-US/00-prerequisites/02-self-paced#iam-policy-for-bedrock-agentcore-workshop) を追加します。

In [ ]:
# Note: Uncomment and run only for self-paced labs
# !aws sts get-caller-identity 

**ステップ 0.2: [セルフペースラボのみ]**

この `prereq.sh` スクリプトに必要なすべての権限を取得したら、以下のコマンドを実行して CloudFormation テンプレートをデプロイします。

In [ ]:
# Note: Uncomment and run only for self-paced labs
# !bash scripts/prereq.sh

### ステップ 1: 依存関係のインストールとライブラリのインポート
始める前に、このLabに必要な依存関係をインストールしましょう。

In [ ]:
# Install required packages
%pip install -U -r requirements.txt -q

必要なライブラリをインポートし、boto3 セッションを初期化します。

In [ ]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool

In [ ]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

### ステップ 2: カスタムツールの実装

次に、カスタマーサポートエージェントに提供する3つのツールを実装します。

Strands Agent でツールを定義するのは非常にシンプルです。関数に `@tool` デコレータを追加し、関数のドキュメント文字列にツールの説明を記述するだけです。Strands Agents は、関数のドキュメント、型付け、引数を使用して、このツールに関するコンテキストをエージェントに提供します。

#### ツール 1: 返品ポリシーの取得

**目的:** このツールは、異なる製品カテゴリの返品ポリシーを顧客が理解するのに役立ちます。返品期間、条件、プロセス、返金のタイムラインに関する詳細情報を提供し、顧客が商品を返品する際に何を期待できるかを正確に把握できるようにします。

In [ ]:
@tool
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information for a specific product category.

    Args:
        product_category: Electronics category (e.g., 'smartphones', 'laptops', 'accessories')

    Returns:
        Formatted return policy details including timeframes and conditions
    """
    # Mock return policy database - in real implementation, this would query policy database
    return_policies = {
        "smartphones": {
            "window": "30 days",
            "condition": "Original packaging, no physical damage, factory reset required",
            "process": "Online RMA portal or technical support",
            "refund_time": "5-7 business days after inspection",
            "shipping": "Free return shipping, prepaid label provided",
            "warranty": "1-year manufacturer warranty included",
        },
        "laptops": {
            "window": "30 days",
            "condition": "Original packaging, all accessories, no software modifications",
            "process": "Technical support verification required before return",
            "refund_time": "7-10 business days after inspection",
            "shipping": "Free return shipping with original packaging",
            "warranty": "1-year manufacturer warranty, extended options available",
        },
        "accessories": {
            "window": "30 days",
            "condition": "Unopened packaging preferred, all components included",
            "process": "Online return portal",
            "refund_time": "3-5 business days after receipt",
            "shipping": "Customer pays return shipping under $50",
            "warranty": "90-day manufacturer warranty",
        },
    }

    # Default policy for unlisted categories
    default_policy = {
        "window": "30 days",
        "condition": "Original condition with all included components",
        "process": "Contact technical support",
        "refund_time": "5-7 business days after inspection",
        "shipping": "Return shipping policies vary",
        "warranty": "Standard manufacturer warranty applies",
    }

    policy = return_policies.get(product_category.lower(), default_policy)
    return (
        f"Return Policy - {product_category.title()}:\n\n"
        f"• Return window: {policy['window']} from delivery\n"
        f"• Condition: {policy['condition']}\n"
        f"• Process: {policy['process']}\n"
        f"• Refund timeline: {policy['refund_time']}\n"
        f"• Shipping: {policy['shipping']}\n"
        f"• Warranty: {policy['warranty']}"
    )


print("✅ 返品ポリシーツール準備完了")

#### ツール 2: 製品情報の取得

**目的:** このツールは、保証、利用可能なモデル、主要な機能、配送ポリシー、返品情報など、包括的な製品詳細を顧客に提供します。顧客が十分な情報に基づいて購入を決定し、購入する製品について理解するのに役立ちます。

In [ ]:
@tool
def get_product_info(product_type: str) -> str:
    """
    Get detailed technical specifications and information for electronics products.

    Args:
        product_type: Electronics product type (e.g., 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        Formatted product information including warranty, features, and policies
    """
    # Mock product catalog - in real implementation, this would query a product database
    products = {
        "laptops": {
            "warranty": "1-year manufacturer warranty + optional extended coverage",
            "specs": "Intel/AMD processors, 8-32GB RAM, SSD storage, various display sizes",
            "features": "Backlit keyboards, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux support varies by model",
            "support": "Technical support and driver updates included",
        },
        "smartphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "5G/4G connectivity, 128GB-1TB storage, multiple camera systems",
            "features": "Wireless charging, water resistance, biometric security",
            "compatibility": "iOS/Android, carrier unlocked options available",
            "support": "Software updates and technical support included",
        },
        "headphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "Wired/wireless options, noise cancellation, 20Hz-20kHz frequency",
            "features": "Active noise cancellation, touch controls, voice assistant",
            "compatibility": "Bluetooth 5.0+, 3.5mm jack, USB-C charging",
            "support": "Firmware updates via companion app",
        },
        "monitors": {
            "warranty": "3-year manufacturer warranty",
            "specs": "4K/1440p/1080p resolutions, IPS/OLED panels, various sizes",
            "features": "HDR support, high refresh rates, adjustable stands",
            "compatibility": "HDMI, DisplayPort, USB-C inputs",
            "support": "Color calibration and technical support",
        },
    }
    product = products.get(product_type.lower())
    if not product:
        return f"Technical specifications for {product_type} not available. Please contact our technical support team for detailed product information and compatibility requirements."

    return (
        f"Technical Information - {product_type.title()}:\n\n"
        f"• Warranty: {product['warranty']}\n"
        f"• Specifications: {product['specs']}\n"
        f"• Key Features: {product['features']}\n"
        f"• Compatibility: {product['compatibility']}\n"
        f"• Support: {product['support']}"
    )


print("✅ 製品情報取得ツール準備完了")

#### ツール 3: ウェブ検索

**目的:** このツールにより、顧客はトラブルシューティングのサポートや製品の推奨事項などを取得できます。

In [ ]:
@tool
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.

    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.

    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"


print("✅ ウェブ検索ツール準備完了")

#### カスタマーサポートエージェント - Knowledge Base 統合ステップ

##### S3 から製品テクニカルサポートファイルをダウンロード

In [ ]:
import os


def download_files():
    # Get account and region
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    region = boto3.Session().region_name
    bucket_name = f"{account_id}-{region}-kb-data-bucket"

    # Create local folder
    os.makedirs("knowledge_base_data", exist_ok=True)

    # Download all files
    s3 = boto3.client("s3")
    objects = s3.list_objects_v2(Bucket=bucket_name)

    for obj in objects["Contents"]:
        file_name = obj["Key"]
        s3.download_file(bucket_name, file_name, f"knowledge_base_data/{file_name}")
        print(f"ダウンロード完了: {file_name}")

    print("すべてのファイルを保存しました: knowledge_base_data/")


# Run it
download_files()

#### Knowledge Base 同期ジョブ

##### S3 からの製品テクニカルサポートファイルで Knowledge Base を同期し、エージェントと統合できるようにする

In [ ]:
import time

# Get parameters
ssm = boto3.client("ssm")
bedrock = boto3.client("bedrock-agent")
s3 = boto3.client("s3")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")[
    "Parameter"
]["Value"]
ds_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/data-source-id")[
    "Parameter"
]["Value"]

# Get file names from S3 bucket
bucket_name = f"{account_id}-{region}-kb-data-bucket"
s3_objects = s3.list_objects_v2(Bucket=bucket_name)
file_names = [obj["Key"] for obj in s3_objects.get("Contents", [])]

# Start sync job
response = bedrock.start_ingestion_job(
    knowledgeBaseId=kb_id, dataSourceId=ds_id, description="Quick sync"
)

job_id = response["ingestionJob"]["ingestionJobId"]
print("Bedrock Knowledge Base 同期ジョブを開始しました。S3 からデータファイルを取り込んでいます")

# Monitor until complete
while True:
    job = bedrock.get_ingestion_job(
        knowledgeBaseId=kb_id, dataSourceId=ds_id, ingestionJobId=job_id
    )["ingestionJob"]

    status = job["status"]

    if status in ["COMPLETE", "FAILED"]:
        break

    time.sleep(10)

# Print final result
if status == "COMPLETE":
    file_count = job.get("statistics", {}).get("numberOfDocumentsScanned", 0)
    files_list = ", ".join(file_names)
    print(
        f"Bedrock Knowledge Base 同期ジョブが正常に完了しました。{file_count} 件のファイルを取り込みました"
    )
    print(f"取り込まれたファイル: {files_list}")
else:
    print(f"Bedrock Knowledge Base 同期ジョブが失敗しました。ステータス: {status}")

#### ツール 4: テクニカルサポートの取得

**目的:** このツールは、電子機器ドキュメントの Knowledge Base にアクセスして、顧客に包括的なテクニカルサポートとトラブルシューティング支援を提供します。詳細なセットアップガイド、メンテナンス手順、トラブルシューティングステップ、接続ソリューション、保証サービス情報が含まれています。このツールは、顧客が技術的な問題を解決し、デバイスを適切に構成し、製品の最適なパフォーマンスのためのメンテナンス要件を理解するのに役立ちます。

In [ ]:
from strands.models import BedrockModel
from strands import Agent
from strands_tools import retrieve


@tool
def get_technical_support(issue_description: str) -> str:
    try:
        # Get KB ID from parameter store
        ssm = boto3.client("ssm")
        account_id = boto3.client("sts").get_caller_identity()["Account"]
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")[
            "Parameter"
        ]["Value"]
        print(f"KB ID を正常に取得しました: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "tech_support_query",
            "input": {
                "text": issue_description,
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4,
            },
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access technical support documentation. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"get_technical_support の詳細エラー: {str(e)}")
        return f"Unable to access technical support documentation. Error: {str(e)}"


print("✅ テクニカルサポートツール準備完了")

### ステップ 4: カスタマーサポートエージェントの作成と設定

次に、モデル、前のステップで実装したツールのリスト、およびシステムプロンプトを提供して、カスタマーサポートエージェントを作成します。

In [ ]:
SYSTEM_PROMPT = """あなたは電子機器 e コマース会社の親切でプロフェッショナルなカスタマーサポートアシスタントです。
あなたの役割は：
- 利用可能なツールを使用して正確な情報を提供する
- 技術情報、製品仕様、メンテナンスに関する質問で顧客をサポートする
- 顧客に対してフレンドリーで忍耐強く、理解を示す
- 質問に回答した後は常に追加のサポートを提供する
- 対応できない場合は、適切な連絡先に案内する

以下のツールにアクセスできます：
1. get_return_policy() - 保証と返品ポリシーに関する質問用
2. get_product_info() - 特定の製品に関する情報を取得
3. web_search() - 最新の技術ドキュメントや更新情報にアクセス
4. get_technical_support() - トラブルシューティング、セットアップガイド、メンテナンスのヒント、詳細な技術支援用
技術的な問題、セットアップの質問、メンテナンスに関する懸念事項については、包括的な技術ドキュメントとステップバイステップのガイドを含む get_technical_support() ツールを常に使用してください。

電子製品や仕様について推測するのではなく、常に適切なツールを使用して正確で最新の情報を取得してください。"""

# Bedrock モデルの初期化（Anthropic Claude 3.7 Sonnet）
model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=region,
)

# すべてのツールを備えたカスタマーサポートエージェントを作成
agent = Agent(
    model=model,
    tools=[
        get_product_info,  # ツール 1: シンプルな製品情報検索
        get_return_policy,  # ツール 2: シンプルな返品ポリシー検索
        web_search,  # ツール 3: 更新情報のためのウェブアクセス
        get_technical_support,  # ツール 4: テクニカルサポートとトラブルシューティング
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("✅ カスタマーサポートエージェントが正常に作成されました！")

### ステップ 5: カスタマーサポートエージェントのテスト

サンプルクエリでエージェントをテストして、すべてのツールが正しく動作することを確認しましょう。

#### 返品確認のテスト

In [ ]:
response = agent("What's the return policy for my thinkpad X1 Carbon?")

In [ ]:
response = agent("My laptop won't turn on, what should I check?")

#### トラブルシューティングのテスト

In [ ]:
response = agent(
    "I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?"
)

## Lab 1 完了！

機能するカスタマーサポートエージェントプロトタイプを正常に作成しました！達成したこと：

- 3つのカスタムツール（返品ポリシー、製品情報、ウェブ検索）を備えたエージェントを構築
- マルチツールのインタラクションとウェブ検索機能をテスト
- 本番環境への移行の基盤を確立

### 現在の制限事項（これから修正します！）
- **単一ユーザーの会話Memory** - ローカルの会話セッションのため、複数の顧客には複数のセッションが必要
- **会話履歴がセッションに限定** - 長期Memoryやクロスセッション情報が会話で利用できない
- **ツールの再利用性** - ツールが異なるエージェント間で再利用できない
- **ローカルでのみ実行** - スケーラブルではない
- **Identity** - ユーザーやエージェントのアイデンティティやアクセス制御がない
- **オブザーバビリティ** - エージェントの動作への可視性が限定的
- **既存の API** - 顧客データ用の既存のエンタープライズ API へのアクセスがない

##### 次のステップ [Lab 2: Memory を追加してエージェントをパーソナライズ →](lab-02-agentcore-memory.ipynb)